# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [13]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [14]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@arthur_juan***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [15]:
#    API KEY: ZpsQt4djTkQaRfHdIvwM4OVEQ
#    API Scret: qnAxY4HpajivQq76Gmz5uzks4Dp5AwMgZh5wopLI0vOYzEZs83
#
#    Access Token: 904543009957715968-zw2LsSLpUxRXol9z6FxatNjUAT01Wbh
#    Access Token Secret: dPULw1D3Q0fvF6CntlbTFvDdn3jO7MSK8e3ZiLfbNKdEJ

In [16]:
 #Dados de autenticação do twitter:

 #Coloque aqui o identificador da conta no twitter: @arthur_juan

 #leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

 #Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [21]:
 #Produto escolhido:
produto = 'Guardiões da Galáxia'
nome = 'GUARDIOESDG'

 #Quantidade mínima de mensagens capturadas:
n = 500
 #Quantidade mínima de mensagens para a base de treinamento:
t = 300

 #Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [22]:
# #Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [23]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(nome)):
#  
#    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(nome))
     #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)
    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)
     #fecha o arquivo
    writer.save()

In [36]:
import pandas as pd
from collections import Counter
dados = pd.read_excel('Planilha_Guardioes.xlsx')


def format_df(dataframe):
    fm1=[]
    fm2=[]
    df_formated = pd.DataFrame({"Treinamento":[]})
    for i in range(len(dataframe)):
        fm1.append(dataframe['Treinamento'][i].translate({ord(c): None for c in "@:;,[]“\/{}()+=-_*.#"}))
        fm2.append(dataframe['Relevância'][i])
    df_formated["Treinamento"]=fm1
    df_formated["Relevância"]=fm2
    return df_formated

df=format_df(dados)
df


,Treinamento,Relevância
0,gostei de um vídeo youtube httpstcow5bkydu1x8 ...,0
1,irei assistir piratas do caribe e guardiões da...,0
2,soloawakens eu tô é trocando blockbusters por ...,0
3,rt omelete arte conceitual de guardiões da gal...,0
4,rt luizhrique guardiões da galáxia vol 2 2017\...,1
5,guardiões da galáxia vol 2 gamora ainda mais l...,0
6,antes de entrar na minha bad só queria dizer q...,1
7,gostei de um vídeo youtube httpstcohxrpivquml ...,0
8,rt omelete arte conceitual retrata cena emocio...,0
9,a mim o que me estranha é o filmebanda desenha...,1


___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [2]:
import pandas as pd
from collections import Counter
dados = pd.read_excel('GUARDIOES.xlsx')


def format_df(dataframe):
    fm1=[]
    fm2=[]
    df_formated = pd.DataFrame({"Treinamento":[]})
    for i in range(len(dataframe)):
        fm1.append(dataframe['Treinamento'][i].translate({ord(c): None for c in "@:;,[]“\/{}()+=-_*.#"}))
        fm2.append(dataframe['Relevância'][i])
    df_formated["Treinamento"]=fm1
    df_formated["Relevância"]=fm2
    return df_formated

df=format_df(dados)
df

KeyError: 'Relevância'

In [11]:
rel_count=df["Relevância"].value_counts()
print(rel_count)


0    284
1     16
Name: Relevância, dtype: int64


In [18]:
splitted_rel = []
splitted_irel = []
    
def constant_calculator(dataframe):
    #separando os elementos por relevancia
    pal_rel = []
    pal_irel = []
    
    for i in range(len(dados)):
        if df["Relevância"][i] == 0:
            pal_irel.append(df['Treinamento'][i])
        else:
            pal_rel.append(df['Treinamento'][i])
    
    #separando as frases em palavras
    splitted_rel = []
    splitted_irel = []
    
    for i in range(len(pal_rel)):
        splitted_rel += pal_rel[i].split()

    for i in range(len(pal_irel)):
        splitted_irel += pal_irel[i].split()
    
    #calculando a probabilidade da relevancia de uma frase
    prob_rel = (rel_count[1]/len(df))
    prob_irel = (rel_count[0])/len(df)
    
    #contando a frequencia de cada palavra
    freq_rel  = Counter(splitted_rel)
    freq_irel = Counter(splitted_irel)
    
    #contando o numero de palavras em cada caso
    palavrasR = len(splitted_rel)
    palavrasIR = len(splitted_irel)
    
    #contando o numero total de palavras unicas (sem repetiçao)
    freq_total = Counter(splitted_rel + splitted_irel)
    total = len(freq_total)
    
    return [prob_rel,prob_irel,palavrasR,palavrasIR,freq_total,total]

constants=constant_calculator(df)

prob_rel=constants[0]
prob_irel=constants[1]
palavrasR=constants[2]
palavrasIR=constants[3]
freq_total=constants[4]
total = constants[5]

In [19]:
# palavrasR = len(splitted_rel)
# palavrasIR = len(splitted_irel)
# freq_total = Counter(splitted_rel + splitted_irel)
total = len(freq_total)
print(total)
print(palavrasR)
#print(1/(total+palavrasIR))

1142
252


In [20]:
def relevance_checker(sentence):
    sentence=sentence.translate({ord(c): None for c in "@:;,[]“\/{}()+=-_*.#"})
    splitted_sentence=sentence.split()
    prob_word_rel=[prob_rel] #para conseguir analisar a probalilidade de cada palavra separadamente foi criado uma lista
    prob_word_irel=[prob_irel] #
    relevance=1
    irrelevant=1
    
    for i in range(len(splitted_sentence)):
        if splitted_sentence[i] in splitted_rel:
            prob_word_rel.append((freq_rel[splitted_sentence[i]]+1)/(total+palavrasR))
        else:
            prob_word_rel.append(1/(total+palavrasR))
            
        if splitted_sentence[i] in splitted_irel:
            prob_word_irel.append((freq_irel[splitted_sentence[i]]+1)/(total+palavrasIR))
        else:
            prob_word_irel.append(1/(total+palavrasIR))

    
    for i in range (len(prob_word_rel)):
        relevance*=prob_word_rel[i]
        irrelevant*=prob_word_irel[i]
        
    if relevance>irrelevant:
        return "RELEVANTE",relevance
    else:
        return "IRRELEVANTE",irrelevant

relevance_checker("htc vive good ")

('RELEVANTE', 1.9688399207030433e-11)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [21]:
relevante=0
frases_rel=[]
df_teste=pd.read_excel("vive_teste.xlsx")
df_teste["Relevância"].value_counts()
for i in range(len(df_teste)):
    if relevance_checker(df_teste["Teste"][i])[0]=="RELEVANTE" and df_teste["Teste"][i] not in frases_rel :
        relevante+=1
        frases_rel.append(df_teste["Teste"][i])
        
print(relevante)       
print(frases_rel)

161
['spartaga vr - announcement trailer 2017【htc vive, oculus rift】house of how https://t.co/yvirr3aepm', 'these are the htc vive games launching in september https://t.co/lxycstsne6', 'rt @rickking16: watch "tomb raider in virtual reality! | tomb raider #vr: dagger of xian (htc vive gameplay)" on youtube https://t.co/krwtu…', "rt @rtovr: [don't miss] htc vive gets major price cut, now $600 https://t.co/gvxsabquph @htcvive @agraylin @obriend17 https://t.co/hvxsy3li…", '#let\'splay #thebellows on "htcvive https://t.co/r9tc6atkgg via @youtube #scary\n#horror #creepy #vive #htc #vr #virtualreality #steam #spooky', 'face-off against a giant ice dragon in @htcvive title tales of glacier - https://t.co/2txzlpb6jn https://t.co/kcn3jvilon', 'mindshow brings vr puppet theater to steam’s early access for htc vive |  https://t.co/uavioow4hg https://t.co/end9acuzrv', 'rt @shahidkamal: chimera zero. for psvr, oculus rift and htc vive. tech demo currently runs on psvr and rift. this is not your dad

In [23]:
for i in range(len(frases_rel)):
    print(frases_rel[i])

spartaga vr - announcement trailer 2017【htc vive, oculus rift】house of how https://t.co/yvirr3aepm
these are the htc vive games launching in september https://t.co/lxycstsne6
rt @rickking16: watch "tomb raider in virtual reality! | tomb raider #vr: dagger of xian (htc vive gameplay)" on youtube https://t.co/krwtu…
rt @rtovr: [don't miss] htc vive gets major price cut, now $600 https://t.co/gvxsabquph @htcvive @agraylin @obriend17 https://t.co/hvxsy3li…
#let'splay #thebellows on "htcvive https://t.co/r9tc6atkgg via @youtube #scary
#horror #creepy #vive #htc #vr #virtualreality #steam #spooky
face-off against a giant ice dragon in @htcvive title tales of glacier - https://t.co/2txzlpb6jn https://t.co/kcn3jvilon
mindshow brings vr puppet theater to steam’s early access for htc vive |  https://t.co/uavioow4hg https://t.co/end9acuzrv
rt @shahidkamal: chimera zero. for psvr, oculus rift and htc vive. tech demo currently runs on psvr and rift. this is not your daddy’s chim…
htc shares rebound

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
